**Table of contents**<a id='toc0_'></a>    
- [Prophet Model](#toc1_1_)    
    - [Prophet Tuned Model](#toc1_1_1_)    
    - [Facebook Prophet Analysis](#toc1_1_2_)    
      - [Preparing Data for Prophet](#toc1_1_2_1_)    
    - [Recurrent Neural Networks (RNNs)](#toc1_1_3_)    
      - [Data Preprocessing for RNNs](#toc1_1_3_1_)    
      - [Building and Training the RNN](#toc1_1_3_2_)    
      - [Predictions and Evaluation](#toc1_1_3_3_)    
- [Code of the dead](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Prophet Model](#toc0_)
- Prophet model expects the dataset to be named a specific way. We will rename our dataframe columns before feeding it into the model.
    - Datetime column named: `ds`
    - target : `y`

In [6]:
from capstone_functions import *
plt.style.use('seaborn')

import warnings
warnings.filterwarnings('ignore')

# Set the random.seed(42)
np.random.seed(42)


Versions used in this notebook:
Python version: 3.9.17 (main, Jul  5 2023, 21:22:06) [MSC v.1916 64 bit (AMD64)]
Pandas version: 2.0.2
Numpy version: 1.23.4
Seaborn version: 0.12.2
Matplotlib version: 3.7.1
Scipy version: 1.10.1
Statsmodels version: 0.14.0
SKLearn version: 1.2.2


C:\Users\Samsickle\AppData\Local\Temp\ipykernel_33444\1022261773.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [7]:
from prophet.plot import plot_cross_validation_metric, plot_plotly
import statsmodels.api as sm
from prophet.serialize import model_to_json, model_from_json
from prophet.diagnostics import cross_validation, performance_metrics
import scipy.stats as stats
import itertools


In [ ]:
master_df = pd.read_parquet("../Data/master_df.parquet")

#prepare the data for prophet
master_df = master_df.reset_index().rename(columns={'date':'ds', 'new_confirmed':'y'}).copy()

master_df.info()
master_df.head()



In [ ]:
no_null_rows = master_df.dropna().copy()

no_null_col = master_df.dropna(axis=1).copy()

cutoff=15
cutoff_date = no_null_col.ds.max() - pd.Timedelta(days=cutoff)

train_df = no_null_col[no_null_col.ds < cutoff_date].copy()
test_df = no_null_col[no_null_col.ds >= cutoff_date].copy()


In [ ]:
y_train = train_df[['ds','y']].copy()
y_test = test_df[['ds','y']].copy()


most_important = ['stringency_index', 'derived_reproduction_rate', 'new_bi_weekly_vaccinations', 
                'current_hospitalized_patients',  'excess_mortality', 'ds']

X_train = train_df[most_important].copy()
X_test = test_df[most_important].copy()

val_eval_holder = pd.DataFrame().rename_axis('Validation Metric', axis=1)
in_sample_eval_holder = pd.DataFrame().rename_axis('In-Sample Metric', axis=1)
out_of_sample_eval_holder = pd.DataFrame().rename_axis('Out-of-Sample Metric', axis=1)


## Base Model of Prophet

In [ ]:
def residual_plotter(forecast_df, yhat_col='yhat', original_col='y_train'):
    """
    Plots residual diagnostics and returns evaluation metrics for a forecast dataframe from Prophet.
    
    Parameters:
    - forecast_df: DataFrame from Prophet prediction.
    
    Returns:
    - scores: Series with R2, MAE, RMSE, and SMAPE scores.
    """
    
    # Calculate residuals and lags
    residuals = (forecast_df[original_col] - forecast_df[yhat_col]).dropna()
    lags = int(min(20, round(len(residuals)/3) - 1))
    
    fig, ax = plt.subplots(2, 2, figsize=(12, 8))

    # Plot Histogram
    sns.histplot(residuals, kde=True,  ax=ax[0,0])
    ax[0,0].set_title("Histogram of Residuals")
    ax[0,0].set_xlabel("Residual")
    ax[0,0].set_ylabel("Frequency")
    ax[0,0].grid(True, which='both', linestyle='--', linewidth=0.5)
    
    # QQ Plot
    stats.probplot(residuals, dist="norm", plot=ax[0,1])
    ax[0,1].set_title("Q-Q Plot of Residuals")
    ax[0,1].grid(True, which='both', linestyle='--', linewidth=0.5)
    
    # ACF and PACF plots
    sm.graphics.tsa.plot_acf(residuals, lags=lags, ax=ax[1,0])
    sm.graphics.tsa.plot_pacf(residuals, lags=lags, ax=ax[1,1])
    
    ax[1,0].set_title("ACF of Residuals")
    ax[1,1].set_title("PACF of Residuals")
    plt.tight_layout()
    plt.show()
    
        
    # Evaluation Metrics
    r2 = r2_score(forecast_df[original_col], forecast_df[yhat_col])
    mae = mean_absolute_error(forecast_df[original_col], forecast_df[yhat_col])
    rmse = mean_squared_error(forecast_df[original_col], forecast_df[yhat_col], squared=False)
    smape_score = smape(forecast_df[original_col], forecast_df[yhat_col])
    
    scores = pd.Series({
        'R2': r2,
        'MAE': mae,
        'RMSE': rmse,
        'SMAPE': smape_score
    })
    
    return scores #, residuals

# residual_plotter(forecast_df.dropna(subset='y_train'), yhat_col='yhat', original_col='y_train')
# residual_plotter(forecast_df.dropna(subset='y_test'), yhat_col='yhat', original_col='y_test')


In [ ]:
master_df[most_important]

In [ ]:
def forecast_df_complete(model, y_train, y_test, exogenous_df=None):

    prophet_model_df = model.make_future_dataframe(15, freq='D', include_history=True)
    
    if master_df is not None:
        prophet_model_df = prophet_model_df.merge(master_df, on='ds', how='left')

    forecast_df = model.predict(prophet_model_df)
    
    
    forecast_df = forecast_df.merge(y_train, on='ds', how='left').copy()
    forecast_df = forecast_df.merge(y_test, on='ds', how='left', suffixes = ['_train', '_test']).copy()
    
    
    return forecast_df

In [ ]:

model = Prophet().fit(y_train)


forecast_df = forecast_df_complete(model, y_train, y_test)


# Generate the Prophet forecast plot
fig = plot_plotly(model, forecast_df)

# Add the test actual data points
fig.add_trace(go.Scatter(
    x=y_test.ds, 
    y=y_test.y, 
    mode='lines', 
    name='Historical Predictions', 
    line=dict(color='green')
))

# Add titles and labels
fig.update_layout(
    title="Prophet Model Forecast vs Actual Data",
    xaxis_title="Date",
    yaxis_title="Value",
    legend_title="Legend"
)

# Display the plot
fig.show();


In [ ]:

fig = model.plot_components(forecast_df)
fig.suptitle('Seasonal Component Breakdown of Forecasted COVID-19 Cases', y=1.05, fontsize=16)
fig.show();
# Using the function on the sample dataframe


In [ ]:

in_sample_eval_holder['Base Prophet Model'] = residual_plotter(forecast_df.dropna(subset='y_train'), yhat_col='yhat', original_col='y_train')


In [ ]:
val_eval_holder

In [ ]:



df_cv = cross_validation(
    model, 
    initial=f"366 days", 
    period=f"15 days", 
    horizon=f"15 days",
    parallel="processes")

val_eval_holder['Base Prophet Model'] = pd.to_numeric(performance_metrics(df_cv).mean().drop(index='horizon'))

plot_cross_validation_metric(df_cv, metric='rmse')

plt.title('RMSE for COVID-19 Forecasting using Prophet');


In [ ]:


out_of_sample_eval_holder['Base Prophet Model'] = residual_plotter(forecast_df.dropna(subset='y_test'), yhat_col='yhat', original_col='y_test')


# with open('serialized_model.json', 'w') as fout:
#     fout.write(model_to_json(model))  # Save model

# with open('serialized_model.json', 'r') as fin:
#     model = model_from_json(fin.read())  # Load model


In [ ]:
display(val_eval_holder.style.background_gradient(cmap='coolwarm', axis=1))
display(in_sample_eval_holder.style.background_gradient(cmap='coolwarm', axis=1))
# display(out_of_sample_eval_holder.style.background_gradient(cmap='coolwarm', axis=1))

### <a id='toc1_1_1_'></a>[Prophet Tuned Model](#toc0_)

In [ ]:
brute_force = False

In [ ]:
if brute_force:
    # Python
    import itertools

    param_grid = {  
        'changepoint_prior_scale': np.logspace(-2.5,-1.5, 5),
        'seasonality_prior_scale': np.logspace(-1.5,1, 5),
        
    }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here
    mae = []
    smape = []
    coverage = []


    # Use cross validation to evaluate all parameters
    for params in all_params:    
        m = Prophet(**params).fit(y_train)  # Fit model with given params
        
        df_cv = cross_validation(
            m, 
            initial=f"366 days", 
            period=f"15 days", 
            horizon=f"15 days",
            parallel="processes")

        
        df_p = performance_metrics(df_cv, rolling_window=29)
        rmses.append(df_p['rmse'].values[0])
        mae.append(df_p['mae'].values[0])
        smape.append(df_p['smape'].values[0])
        coverage.append(df_p['coverage'].values[0])
        
    # Find the best parameters

    tuning_results1 = pd.DataFrame(all_params)
    tuning_results1['rmse'] = rmses
    tuning_results1['mae'] = mae
    tuning_results1['smape'] = smape
    tuning_results1['coverage'] = coverage


In [ ]:
if brute_force:
    best_params = all_params[np.argmin(rmses)]
    print(best_params)


    tuning_results1['compo'] = (tuning_results['rmse'] + tuning_results1['mae']) * tuning_results1['smape']

    tuning_results1.sort_values(by='changepoint_prior_scale').style.background_gradient(cmap='coolwarm', subset=['smape','mae', 'rmse', 'compo'])

In [ ]:
if brute_force:
    param_grid = {  
        'changepoint_prior_scale': np.logspace(-2, -1, 7),
        'seasonality_prior_scale': np.logspace(-2, 2, 7),
    }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here
    mae = []
    smape = []
    coverage = []

    # Use cross validation to evaluate all parameters
    for params in all_params:    
        m = Prophet(**params).fit(y_train)  # Fit model with given params
        
        df_cv = cross_validation(
            m, 
            initial=f"366 days", 
            period=f"15 days", 
            horizon=f"15 days",
            parallel="processes")

        df_p = performance_metrics(df_cv, rolling_window=29)
        rmses.append(df_p['rmse'].values[0])
        mae.append(df_p['mae'].values[0])
        smape.append(df_p['smape'].values[0])
        coverage.append(df_p['coverage'].values[0])

    # Find the best parameters

    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses
    tuning_results['mae'] = mae
    tuning_results['smape'] = smape
    tuning_results['coverage'] = coverage



In [ ]:
if brute_force:
    best_params = all_params[np.argmin(rmses)]
    print(best_params)


    tuning_results['compo'] = ((tuning_results['rmse'] + tuning_results['mae']) / 2) / y_train.std()[1]
    tuning_results['compo2'] = (tuning_results['rmse'] + tuning_results['mae']) * tuning_results['smape'] / y_train.std()[1]

    tuning_results.sort_values(by='compo').head(15).style.background_gradient(cmap='coolwarm', subset=['smape','mae', 'rmse', 'compo', 'compo2'])


In [ ]:

model2 = Prophet(changepoint_prior_scale = 0.031623, seasonality_prior_scale = 0.046416)
model2.add_country_holidays(country_name='US')

model2.fit(y_train)
# changepoint_prior_scale = 0.031623
# seasonality_prior_scale = 0.046416

forecast_df = forecast_df_complete(model2, y_train, y_test)


# Generate the Prophet forecast plot
fig = plot_plotly(model2, forecast_df)

# Add the test actual data points
fig.add_trace(go.Scatter(
    x=y_test.ds, 
    y=y_test.y, 
    mode='lines', 
    name='Historical Predictions', 
    line=dict(color='green')
))

# Add titles and labels
fig.update_layout(
    title="Tuned Prophet Model Forecast vs Actual Data",
    xaxis_title="Date",
    yaxis_title="Value",
    legend_title="Legend"
)

# Display the plot
fig.show();


In [ ]:

fig = model2.plot_components(forecast_df)
fig.suptitle('Seasonal Component Breakdown of Forecasted COVID-19 Cases', y=1.05, fontsize=16)
fig.show();
# Using the function on the sample dataframe


In [ ]:

in_sample_eval_holder['Tuned Prophet Model'] = residual_plotter(forecast_df.dropna(subset='y_train'), yhat_col='yhat', original_col='y_train')


In [ ]:

df_cv = cross_validation(
    model2, 
    initial=f"366 days", 
    period=f"15 days", 
    horizon=f"15 days",
    parallel="processes")

val_eval_holder['Tuned Prophet Model'] = pd.to_numeric(performance_metrics(df_cv).mean().drop(index='horizon'))

plot_cross_validation_metric(df_cv, metric='rmse')

plt.title('RMSE for COVID-19 Forecasting using Prophet');


In [ ]:


out_of_sample_eval_holder['Tuned Prophet Model'] = residual_plotter(forecast_df.dropna(subset='y_test'), yhat_col='yhat', original_col='y_test')


# with open('serialized_model.json', 'w') as fout:
#     fout.write(model_to_json(model))  # Save model

# with open('serialized_model.json', 'r') as fin:
#     model = model_from_json(fin.read())  # Load model


In [ ]:
display(val_eval_holder.style.background_gradient(cmap='coolwarm', axis=1))
display(in_sample_eval_holder.style.background_gradient(cmap='coolwarm', axis=1))
# display(out_of_sample_eval_holder.style.background_gradient(cmap='coolwarm', axis=1))


### <a id='toc1_1_2_'></a>[Facebook Prophet Analysis](#toc0_)

Facebook Prophet is designed for forecasting time series data. Let's see how we can use it.


In [ ]:

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.feature_selection import RFE


In [ ]:
exog_train = pd.merge(X_train, y_train, on = ['ds'])

exog_test = pd.merge(X_test, y_test, on = ['ds'])

X_train_full = train_df.drop(columns=['y','ds'])


In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(X_train_full):
    # Drop columns: 'current_hospitalized_patients', 'current_intensive_care_patients' and 20 other columns
    columns_to_keep = ['current_hospitalized_patients', 'current_intensive_care_patients', 'cancel_events', 'derived_reproduction_rate', 
                'excess_mortality', 'gatherings_restrictions', 'information_campaigns', 'internal_movement_restrictions', 'international_movement_restrictions', 
                'school_closing', 'sig_recovery_rate', 'stay_home_restrictions', 'stringency_index', 'testing_policy', 'transport_closing', 'workplace_closing', 
                'dayofweek', 'quarter', 'month', 'new_bi_weekly_vaccinations', 'fatal', 'kap_mortality_rate']
    
    
    X_train_full = X_train_full.loc[:, columns_to_keep]
    return X_train_full


# 'susceptible'

X_train_full = clean_data(X_train_full.copy())
X_train_full.head()

In [ ]:
X_train_full.columns

X_train_full = add_constant(X_train_full)

#  VIF values
vif_data = pd.DataFrame()
vif_data["Feature"] = X_train_full.columns

# Calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X_train_full.values, i) for i in range(X_train_full.shape[1])]
display(vif_data.sort_values(by='VIF', ascending=False).style.background_gradient(cmap='coolwarm', vmax=10))


X_train_full.drop(columns=['const'], inplace=True)

In [ ]:
if brute_force:
    # Python

    param_grid = {  
        'changepoint_prior_scale': np.logspace(-2.5,-1.5, 4),
        'seasonality_prior_scale': np.logspace(-1.5,1, 4),
        'holidays_prior_scale': np.logspace(-2,1, 4),
    }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here
    mae = []
    smape = []
    coverage = []


    # Use cross validation to evaluate all parameters
    for params in tqdm(all_params):    
        m = Prophet(**params) # Fit model with given params
        m.add_country_holidays(country_name='US')

        for i in X_train_full.columns:
            m.add_regressor(i, prior_scale = .5, standardize='auto', mode='additive')

        m.fit(pd.concat([X_train_full, y_train], axis=1))

        df_cv = cross_validation(
            m, 
            initial=f"366 days", 
            period=f"15 days", 
            horizon=f"15 days",
            parallel="processes")

        
        df_p = performance_metrics(df_cv, rolling_window=29)
        rmses.append(df_p['rmse'].values[0])
        mae.append(df_p['mae'].values[0])
        smape.append(df_p['smape'].values[0])
        coverage.append(df_p['coverage'].values[0])
        
    # Find the best parameters

    tuning_results1 = pd.DataFrame(all_params)
    tuning_results1['rmse'] = rmses
    tuning_results1['mae'] = mae
    tuning_results1['smape'] = smape
    tuning_results1['coverage'] = coverage

    tuning_results1

    best_params = all_params[np.argmin(rmses)]
    print(best_params)


    tuning_results1['compo'] = (tuning_results1['rmse'] + tuning_results1['mae']) * tuning_results1['smape']

    tuning_results1.sort_values(by='rmse').style.background_gradient(cmap='coolwarm', subset=['smape','mae', 'rmse', 'compo', 'coverage'])

In [ ]:
# model3 = Prophet(changepoint_prior_scale = 0.031623, seasonality_prior_scale = 0.046416)

# for i in X_train.columns[:-2]:
#     model3.add_regressor(i, prior_scale=0.5, mode='additive')

# model3.fit(exog_train)



# title= 'Prophet Tuned and Sparse Exogenous Model 1'

In [ ]:
model3 = Prophet(changepoint_prior_scale = 0.031623, seasonality_prior_scale = 0.046416, growth='flat')
model3.add_country_holidays(country_name='US')

for i in X_train_full.columns:
    model3.add_regressor(i, prior_scale = .5, standardize='auto', mode='additive')

model3.fit(pd.concat([X_train_full, y_train], axis=1))


title= 'Prophet Tuned and Big Exogenous Model 2'

In [ ]:
# big_ready = pd.concat([y_train, X_train_full], axis=1).copy()
# big_ready.iloc[:,1:] = big_ready.iloc[:,1:].apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna(axis=0, how='any').copy()

# model3 = Prophet(changepoint_prior_scale = 0.003162, seasonality_prior_scale = 0.215443, holidays_prior_scale = 0.100000, growth='flat')
# model3.add_country_holidays(country_name='US')

# for i in big_ready.columns[2:]:
#     model3.add_regressor(i, prior_scale = .5, standardize='auto', mode='additive')

# model3.fit(big_ready)

# title = 'Prophet Super Tuned and Big Exogenous Model 3'

In [ ]:

forecast_df = forecast_df_complete(model3, y_train, y_test, X_train_full)


# Generate the Prophet forecast plot
fig = plot_plotly(model3, forecast_df)

# Add the test actual data points
fig.add_trace(go.Scatter(
    x=y_test.ds, 
    y=y_test.y, 
    mode='lines', 
    name='Historical Predictions', 
    line=dict(color='green')
))

# Add titles and labels
fig.update_layout(
    title=f"{title} Forecast vs Actual Data",
    xaxis_title="Date",
    yaxis_title="Value",
    legend_title="Legend"
)

# Display the plot
fig.show();


fig = model3.plot_components(forecast_df)
fig.suptitle('Seasonal Component Breakdown of Forecasted COVID-19 Cases', y=1.05, fontsize=16)
fig.show();



# Using the function on the sample dataframe

in_sample_eval_holder[f'{title} Prophet Model'] = residual_plotter(forecast_df.dropna(subset='y_train'), yhat_col='yhat', original_col='y_train')



df_cv = cross_validation(
    model3, 
    initial=f"366 days", 
    period=f"15 days", 
    horizon=f"15 days",
    parallel="processes")

val_eval_holder[f'{title} Prophet Model'] = pd.to_numeric(performance_metrics(df_cv).mean().drop(index='horizon'))

plot_cross_validation_metric(df_cv, metric='rmse')

plt.title(f'RMSE for COVID-19 Forecasting using {title} Prophet Model');



out_of_sample_eval_holder[f'{title} Prophet Model'] = residual_plotter(forecast_df.dropna(subset='y_test'), yhat_col='yhat', original_col='y_test')


# with open('serialized_model.json', 'w') as fout:
#     fout.write(model_to_json(model3))  # Save model3

# with open('serialized_model.json', 'r') as fin:
#     model3 = model_from_json(fin.read())  # Load model3


In [ ]:

display(val_eval_holder.style.background_gradient(cmap='coolwarm', axis=1))
display(in_sample_eval_holder.style.background_gradient(cmap='coolwarm', axis=1))
display(out_of_sample_eval_holder.style.background_gradient(cmap='coolwarm', axis=1))